<a href="https://colab.research.google.com/github/StonecoldLi/Kaggle_Contest_Samli/blob/main/Natural%20Language%20Processing%20with%20Disaster%20Tweets/Bert_finetune_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

读取竞赛数据集

In [ ]:
!pip install transformers tqdm boto3 requests regex -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.4 MB/s eta 0:00:00


In [ ]:
#上传凭证
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jfli318","key":"475ac384f8639d8bb024bc7891fec449"}'}

In [ ]:
#create a kaggle folder
!mkdir ~/.kaggle

In [ ]:
#copy the kaggle.json to folder created
!cp kaggle.json ~/.kaggle/

In [ ]:
# permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v .

- path is now set to: .


In [ ]:
!kaggle competitions download -c nlp-getting-started

100% 593k/593k [00:00<00:00, 634kB/s]
100% 593k/593k [00:00<00:00, 634kB/s]


In [ ]:
!unzip '/content/competitions/nlp-getting-started/nlp-getting-started.zip' -d '.'

Archive:  /content/competitions/nlp-getting-started/nlp-getting-started.zip
  inflating: ./sample_submission.csv  
  inflating: ./test.csv              
  inflating: ./train.csv             


操作

In [ ]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-uncased"

#取得此预训练模型的tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("torch version", torch.__version__)

torch version 2.0.1+cu118


In [ ]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 30522


Finetune

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv("/content/train.csv")
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df_train.fillna('unknown',inplace=True)

In [ ]:
def combine_features(df):
    df['new_text'] = df.keyword.str.cat([df.location,df.text], sep=' ', na_rep='na')
    df.drop(columns = ['keyword','location','id','text'],inplace=True)
    #df.drop(columns='id',inplace=True)
    return df

In [ ]:
df_train1 = combine_features(df_train)
df_train1['label'] = df_train1.target
df_train1.drop(columns=['target'],inplace=True)
df_train1.head()

,new_text,label
0,unknown unknown Our Deeds are the Reason of th...,1
1,unknown unknown Forest fire near La Ronge Sask...,1
2,unknown unknown All residents asked to 'shelte...,1
3,"unknown unknown 13,000 people receive #wildfir...",1
4,unknown unknown Just got sent this photo from ...,1


In [ ]:
df_train1.shape

(7613, 2)

In [ ]:
# # 剔除过长的样本避免爆GPU
# MAX_LENGTH = 40
# df_train1 = df_train1[~(df_train.new_text.apply(lambda x : len(x)) > MAX_LENGTH)]

In [ ]:
# 另存成 tsv 供 PyTorch 使用
df_train1.to_csv("train.tsv", sep="\t", index=False)

In [ ]:
#baseline null_accuracy
df_train1.label.value_counts() / len(df_train)

0    0.57034
1    0.42966
Name: label, dtype: float64

In [ ]:
#训练样本
df_test = pd.read_csv("test.csv")
df_test.head() 

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
def combine_features1(df):
    df['new_text'] = df.keyword.str.cat([df.location,df.text], sep=' ', na_rep='na')
    df.drop(columns = ['keyword','location','text'],inplace=True)
    return df

In [ ]:
df_test.fillna('unknown',inplace=True)
df_test1 = combine_features1(df_test)
df_test1

,id,new_text
0,0,unknown unknown Just happened a terrible car c...
1,2,unknown unknown Heard about #earthquake is dif...
2,3,unknown unknown there is a forest fire at spot...
3,9,unknown unknown Apocalypse lighting. #Spokane ...
4,11,unknown unknown Typhoon Soudelor kills 28 in C...
...,...,...
3258,10861,unknown unknown EARTHQUAKE SAFETY LOS ANGELES ...
3259,10865,unknown unknown Storm in RI worse than last hu...
3260,10868,unknown unknown Green Line derailment in Chica...
3261,10874,unknown unknown MEG issues Hazardous Weather O...


In [ ]:
df_test1.to_csv("test.tsv", sep='\t', index=False)

In [ ]:
print("test_lenth：", len(df_test))
df_test.head()

test_lenth： 3263


,new_text
0,unknown unknown Just happened a terrible car c...
1,unknown unknown Heard about #earthquake is dif...
2,unknown unknown there is a forest fire at spot...
3,unknown unknown Apocalypse lighting. #Spokane ...
4,unknown unknown Typhoon Soudelor kills 28 in C...


将原始文本转化成BERT相容的输入格式

實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None

In [ ]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
 
    
class DisasterDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {int('0'): 0, int('1'): 1}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            #text = self.df.iloc[idx, :1].values
            text = self.df.iloc[idx, 1]
            label_tensor = None
        else:
            text, label = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        len_a = len(word_pieces)
        
        # # 第二個句子的 BERT tokens
        # tokens_b = self.tokenizer.tokenize(text_b)
        # word_pieces += tokens_b + ["[SEP]"]
        # len_b = len(word_pieces) - len_a

        # # 第三個句子的 BERT tokens
        # tokens_c = self.tokenizer.tokenize(text_c)
        # word_pieces += tokens_c + ["[SEP]"]
        # len_c = len(word_pieces) - len_b - len_a #6 注意！！！
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a, 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len

    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = DisasterDataset("train", tokenizer=tokenizer)

In [ ]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
text, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子  ：{text}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子  ：unknown unknown Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
分類  ：1

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([  101,  4242,  4242,  2256, 15616,  2024,  1996,  3114,  1997,  2023,
         1001,  8372,  2089, 16455,  9641,  2149,  2035,   102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

label_tensor   ：1

--------------------

[還原 tokens_tensors]
[CLS]unknownunknownourdeedsarethereasonofthis#earthquakemayallahforgiveusall[SEP]



DataLoader

In [ ]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `FakeNewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids



# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [ ]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([64, 74]) 
tensor([[  101,  4242,  4242,  ...,     0,     0,     0],
        [  101,  4242,  4242,  ...,     0,     0,     0],
        [  101,  4242,  4242,  ...,     0,     0,     0],
        ...,
        [  101, 11113, 24472,  ...,     0,     0,     0],
        [  101, 11113, 24472,  ...,     0,     0,     0],
        [  101, 11113, 24472,  ...,     0,     0,     0]])
------------------------
segments_tensors.shape = torch.Size([64, 74])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([64, 74])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------

在 BERT 之上加入新 layer 成下游任務模型

In [ ]:
# 載入一個可以做中文多分類任務的模型，n_class = 2
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0
classification acc: 0.5815053198476291


训练模型？

In [ ]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 10  # 幸運數字
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    

[epoch 1] loss: 62.587, acc: 0.829
[epoch 2] loss: 46.897, acc: 0.861
[epoch 3] loss: 43.294, acc: 0.873
[epoch 4] loss: 37.127, acc: 0.862
[epoch 5] loss: 35.990, acc: 0.905
[epoch 6] loss: 32.739, acc: 0.914
[epoch 7] loss: 28.366, acc: 0.932
[epoch 8] loss: 22.198, acc: 0.947
[epoch 9] loss: 17.577, acc: 0.961
[epoch 10] loss: 14.229, acc: 0.964
CPU times: user 19min 25s, sys: 3.44 s, total: 19min 29s
Wall time: 19min 53s


In [ ]:
%%time
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = DisasterDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=256, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
predictions = get_predictions(model, testloader)

# 用來將預測的 label id 轉回 label 文字
index_map = {v: k for k, v in testset.label_map.items()}

# 生成 Kaggle 繳交檔案
df = pd.DataFrame({"Category": predictions.tolist()})
df['Category'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, ["id"]], 
                          df.loc[:, 'Category']], axis=1)
df_pred.to_csv('bert_1_prec_training_samples.csv', index=False)
df_pred.head()

CPU times: user 14.8 s, sys: 41 ms, total: 14.8 s
Wall time: 14.8 s


,id,Category
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
trainset

0.81826